In [4]:
import numpy as np
import xarray as xr
import dfm_tools as dfmt

/home/meggeraeds/.conda/envs/hires_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import glob
import warnings

In [38]:
from dask_jobqueue import SLURMCluster
from dask.distributed import Client

In [42]:
n_cores = 1
n_processes = 4
mem_lim = str(int(np.floor(768/(n_cores*n_processes))))+'GB' # 768 GB of memory for one entire memory node on DelftBlue

In [43]:
cluster = SLURMCluster(name='dask-cluster',
                       cores=n_cores,
                       memory=mem_lim,
                       processes=n_processes,
                       interface='ib0',
                       queue='memory',
                       walltime='04:00:00',
                       asynchronous=0)

/home/meggeraeds/.conda/envs/hires_env/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 35719 instead
  warnings.warn(


In [44]:
print(cluster.job_script())

#!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -p memory
#SBATCH -n 1
#SBATCH --cpus-per-task=1
#SBATCH --mem=179G
#SBATCH -t 04:00:00

/home/meggeraeds/.conda/envs/hires_env/bin/python -m distributed.cli.dask_worker tcp://172.21.12.3:39725 --name dummy-name --nthreads 0 --memory-limit 44.70GiB --nworkers 4 --nanny --death-timeout 60 --interface ib0



In [45]:
cluster.scale(1)

In [54]:
client = Client(cluster)

In [86]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: http://172.21.12.3:35719/status,
Dashboard: http://172.21.12.3:35719/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://172.21.12.3:39725,Workers: 0
Dashboard: http://172.21.12.3:35719/status,Total threads: 0
Started: 1 hour ago,Total memory: 0 B


In [79]:
input_regex = r'/scratch/meggeraeds/test_raw/RMM_dflowfm_2022_jul21_aug7_sm_000*.nc'

In [80]:
## 1. Input tests
# 1.x. Loop through all of the files and write when there is something wrong with the file
file_list = glob.glob(input_regex)

# 1.x Make a list for corrupted files
corrupted = []

for file in file_list:

    file_name = file.split('/')[-1]
    
    try:
        test = xr.open_dataset(file)
    except:
        warnings.warn(f'The file {file_name} seems to be corrupted. Check your inputs.', UserWarning)
        corrupted.append(file)

In [81]:
# I.x Check if there are any corrupted files in the listed files
if len(corrupted) > 0:
    print('There are corrupted files. These will be taken out of the file list for the next tests.')
# I.x Get the entire input list and remove the corrupted files
input_files = [f for f in glob.glob(input_regex) if not f in corrupted]

In [82]:
input_files

['/scratch/meggeraeds/test_raw/RMM_dflowfm_2022_jul21_aug7_sm_0006_map.nc',
 '/scratch/meggeraeds/test_raw/RMM_dflowfm_2022_jul21_aug7_sm_0002_map.nc',
 '/scratch/meggeraeds/test_raw/RMM_dflowfm_2022_jul21_aug7_sm_0003_map.nc',
 '/scratch/meggeraeds/test_raw/RMM_dflowfm_2022_jul21_aug7_sm_0005_map.nc',
 '/scratch/meggeraeds/test_raw/RMM_dflowfm_2022_jul21_aug7_sm_0004_map.nc',
 '/scratch/meggeraeds/test_raw/RMM_dflowfm_2022_jul21_aug7_sm_0001_map.nc',
 '/scratch/meggeraeds/test_raw/RMM_dflowfm_2022_jul21_aug7_sm_0008_map.nc',
 '/scratch/meggeraeds/test_raw/RMM_dflowfm_2022_jul21_aug7_sm_0009_map.nc',
 '/scratch/meggeraeds/test_raw/RMM_dflowfm_2022_jul21_aug7_sm_0007_map.nc']

In [84]:
data_xr = dfmt.open_partitioned_dataset(input_files, chunks={'time':40})

>> xu.open_dataset() with 9 partition(s): 1 

/home/meggeraeds/.conda/envs/hires_env/lib/python3.11/site-packages/xarray/core/dataset.py:278: UserWarning: The specified chunks separate the stored chunks along dimension "time" starting at index 40. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
Task exception was never retrieved
future: <Task finished name='Task-489372' coro=<Client._gather.<locals>.wait() done, defined at /home/meggeraeds/.conda/envs/hires_env/lib/python3.11/site-packages/distributed/client.py:2208> exception=AllExit()>
Traceback (most recent call last):
  File "/home/meggeraeds/.conda/envs/hires_env/lib/python3.11/site-packages/distributed/client.py", line 2217, in wait
    raise AllExit()
distributed.client.AllExit


KeyboardInterrupt: 